#### Imports

In [28]:
import os
import warnings
from dotenv import load_dotenv
warnings.filterwarnings('ignore')
import mysql.connector as connection 
load_dotenv()

hostname = os.getenv('DB_HOST')
username = os.getenv('DB_USERNAME')
password =  os.getenv('DB_PASSWORD')

#### 1. Setting Connection with Database

In [7]:
#### To check if connection is established with the Database
try:
    mydb=connection.connect(host=hostname,user=username,password=password,use_pure=True)
    print(mydb.is_connected())                                   
    mydb.close()
except Exception as e:
    print(str(e))

True


In [8]:
# Getting the default Databases present in MySQL workbench 

mydb=connection.connect(host=hostname,user=username,password=password,use_pure=True)
query="SHOW DATABASES"         # Writting the query 
cursor=mydb.cursor()           # creating a cursor to execute the queries 
cursor.execute(query)          # Executing the query
print(cursor.fetchall())       # Fetching the query result
mydb.close()

[('canondb',), ('classicmodels',), ('hr_analytics',), ('information_schema',), ('mavenfuzzyfactory',), ('mysql',), ('mysqlsampledb',), ('performance_schema',), ('sys',)]


#### 2. Creating a Database

In [10]:
# creating a new database 
try:
    mydb=connection.connect(host="localhost",user=username,password=password,use_pure=True)
    db_name = 'MyDatabase'
    query= f"create database {db_name}"
    cursor=mydb.cursor()           
    cursor.execute(query)          
    print(f"Database {db_name} created !")
    mydb.close()
except Exception as e:
    print(e)

Database MyDatabase created !


In [4]:
# # Deleting a Database
# query= "drop database MyDatabase"
# cursor=mydb.cursor()           
# cursor.execute(query)          
# print("Database Deleted !")

Database Deleted !


#### 3. Create a table inside a database

In [11]:
# creating table to the database "MyDatabase"
# Dont miss putting the database name in the connect function argument
try:
    mydb=connection.connect(host=hostname,user=username,password=password,
                            database="MyDatabase", use_pure=True)
    ## Check if the database connection is being established
    print(mydb.is_connected())                                   
    mydb.close()
except Exception as e:
    print(str(e))

True


In [12]:
try:
    mydb=connection.connect(host=hostname,user=username,password=password,
                            database="MyDatabase", use_pure=True)     
    table_name = 'Students'                       
    query=f"create table {table_name} (Student_ID int(10) auto_increment primary key,\
            First_Name Varchar(20) ,Last_Name Varchar(20), Score int(3))"
    cursor=mydb.cursor()           
    cursor.execute(query)          
    print(f"Table {table_name} created !")    
    mydb.close()
except Exception as e:
    print(str(e))

Table Students created !


#### 4. Insert Records inside the Table

In [13]:
# Inserting Data into the Table
try:
    mydb=connection.connect(host=hostname,user=username,password=password,
                            database="MyDatabase", use_pure=True)  
    query1="Insert into Students Values(201615001,'Abheek','Nandy',91)"
    query2="Insert into Students Values(201615002,'Anindya','Das',81)"
    query3="Insert into Students Values(201615003,'Abhishek','Roy',87)"
    queries=[query1,query2,query3]
    for i in queries:
        cursor=mydb.cursor()           
        cursor.execute(i)
        mydb.commit()                               
    print("Data Inserted into the table")
    mydb.close()
    
except Exception as e:
    print(str(e))

Data Inserted into the table


#### 5. Fetching records from the table

In [14]:
# Retreiving data from the Database
try:
    mydb=connection.connect(host=hostname,user=username,password=password,
                            database="MyDatabase", use_pure=True) 
    query="Select * from Students"
    cursor=mydb.cursor()           
    cursor.execute(query)
    print(cursor.fetchall())

except Exception as e:
    print(e)

[(201615001, 'Abheek', 'Nandy', 91), (201615002, 'Anindya', 'Das', 81), (201615003, 'Abhishek', 'Roy', 87)]


#### 6. Insert and Query a Whole Dataframe into MYSQL database

In [21]:
# Step1 : Reading the csv file and creating a Dataframe  
import pandas as pd 
df=pd.read_csv(".\\CSV\\DatabaseDF.csv")
df.drop("Unnamed: 0",axis=1,inplace=True)
df['District'] = df['District'].apply(lambda x : x.strip())
df['State'] = df['State'].apply(lambda x : x.strip())
df.head(3)

,District,State,Literacy
0,Thane,Maharashtra,84.53
1,North Twenty Four Parganas,West Bengal,84.06
2,Bangalore,Karnataka,87.67


In [15]:
## Step2 : Creating Database to store the data
try:
    mydb=connection.connect(host=hostname,user=username,password=password,use_pure=True)
    query= "create database India"
    cursor=mydb.cursor()           
    cursor.execute(query)          
    print("Database 'India' created !")
    mydb.close()
except Exception as e :
    print(e)

Database 'India' created !


In [22]:
## Step 3: Check Connection with DB 
try:
    mydb=connection.connect(host=hostname,user=username,password=password,
                            database="India", use_pure=True)
    ## Check if the database connection is being established
    print(mydb.is_connected())                                   
    mydb.close()
except Exception as e:
    print(str(e))

True


In [23]:
# Step4 : Create a Table to store the data 
try:
    mydb=connection.connect(host=hostname,user=username,password=password,database="India", use_pure=True)
    cursor=mydb.cursor()
    query="create table Literacy (ID int(10) auto_increment primary key, \
           District Varchar(50) ,State Varchar(30), Literacy float(5))"
    cursor.execute(query)          
    print("Table 'Literacy' created !")
    mydb.close()

except Exception as e:
    print(e)

Table 'Literacy' created !


In [24]:
## Step 5 : Inserting data into the Table
try:
    mydb=connection.connect(host=hostname,user=username,password=password,database="India", use_pure=True)
    cursor=mydb.cursor()

    colvalues=[]
    for i,row in df.iterrows():
        colvalues.append(tuple([i+1]+list(row)))

    # Insert DataFrame recrds one by one.
    for i in colvalues:
        query = "INSERT INTO Literacy VALUES{}".format(i)
        cursor.execute(query)
        mydb.commit()
        mydb.close()

except Exception as e:
    print(e)

In [29]:
## Step 6 : Retreiving the same data from the MYSQLdatabase 
try:
    mydb=connection.connect(host=hostname,user=username,password=password,database="India", use_pure=True)
    query="Select * from literacy"
    df_=pd.read_sql(query,mydb)
    mydb.close()
except Exception as e:
    print(e)

In [31]:
df_.head(3)

,ID,District,State,Literacy
0,1,Thane,Maharashtra,84.53
1,2,North Twenty Four Parganas,West Bengal,84.06
2,3,Bangalore,Karnataka,87.67


#### 7. Some other DQL

In [33]:
db=connection.connect(host=hostname,user=username,password=password,database="India", use_pure=True)

In [34]:
query="select State,count(District) as Districts from literacy group by State order by count(District) desc"
df=pd.read_sql(query,db)
df.head(3)

,State,Districts
0,Uttar Pradesh,71
1,Madhya Pradesh,50
2,Bihar,38


In [35]:
query="select State,Avg(Literacy) from literacy group by State order by Avg(Literacy) desc"
df=pd.read_sql(query,db)
df.head(3)

,State,Avg(Literacy)
0,Kerala,93.694999
1,Lakshadweep,91.849998
2,Mizoram,89.361251


In [36]:
query="select State,Avg(Literacy) from literacy group by State order by Avg(Literacy) desc limit 5"
df=pd.read_sql(query,db)
df

,State,Avg(Literacy)
0,Kerala,93.694999
1,Lakshadweep,91.849998
2,Mizoram,89.361251
3,Goa,88.579998
4,Puducherry,87.457502


In [39]:
# Sorting by average Literacy (Descending)
db=connection.connect(host="localhost",database='India',user="root",password="1320",use_pure=True)
cursor=db.cursor()
query="SELECT District from literacy where State='West Bengal'"
cursor.execute(query)
print(cursor.fetchall())


[('North Twenty Four Parganas',), ('South Twenty Four Parganas',), ('Barddhaman',), ('Murshidabad',), ('Paschim Medinipur',), ('Hugli',), ('Nadia',), ('Purba Medinipur',), ('Haora',), ('Kolkata',), ('Maldah',), ('Jalpaiguri',), ('Bankura',), ('Birbhum',), ('Uttar Dinajpur',), ('Puruliya',), ('Koch Bihar',), ('Darjiling',), ('Dakshin Dinajpur',)]


In [43]:
# Printing the District with most literacy in West Bengal 

db=connection.connect(host="localhost",database='India',user="root",password="1320",use_pure=True)
cursor=db.cursor()
query="SELECT District from literacy where State='West Bengal' order by Literacy limit 1"
cursor.execute(query)
print(cursor.fetchall())

[('Uttar Dinajpur',)]


In [42]:
# Print the District and state with Least Literacy Rate 

db=connection.connect(host="localhost",database='India',user="root",password="1320",use_pure=True)
cursor=db.cursor()
query="Select District,State from literacy order by Literacy limit 1"
cursor.execute(query)
print(cursor.fetchall())


[('Alirajpur', 'Madhya Pradesh')]
